# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-188689
aml-quickstarts-188689
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [3]:
# choose a name for experiment
experiment_name = 'heart-automl'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-automl,quick-starts-ws-188689,Link to Azure Machine Learning studio,Link to Documentation


## Create or Attach an AmlCompute cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
data = pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found = False
key = "heart-failure-prediction"
description_text = "Prediction of survival of patients with heart failure"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://github.com/hmza09/nd00333-capstone/blob/master/starter_file/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
                                
# Preview of the first five rows
print(data.head())

# Explore data
print(data.describe())

df = dataset.to_pandas_dataframe()
df.describe()

# Data columns
df.columns = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
x = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df[['DEATH_EVENT']]


    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-automl,AutoML_6a6a1e12-6da0-4973-93fc-74bf858a4609,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion()

{'runId': 'AutoML_6a6a1e12-6da0-4973-93fc-74bf858a4609',
 'target': 'automl-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-03-12T11:50:25.451169Z',
 'endTimeUtc': '2022-03-12T12:15:18.520847Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'automl-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-automl","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-188689","workspace_name":"quick-starts-ws-188689","region":"southcentralus","compute_target":"automl-comp

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: heart-automl,
Id: AutoML_6a6a1e12-6da0-4973-93fc-74bf858a4609_48,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=1.5625, reg_lambda=0.8333333333333334, subsample=0.7, tree_method='auto'))], verbose=False)), ('27', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=7, max_leaves=0, n_estimators=400, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), r

In [12]:
print("Best run metrics :",best_run.get_metrics())

Best run metrics : {'f1_score_macro': 0.8256055249665798, 'AUC_macro': 0.8972312329186993, 'f1_score_weighted': 0.8515375936668004, 'AUC_micro': 0.9101584062779955, 'accuracy': 0.8562192393736018, 'matthews_correlation': 0.6636956776991876, 'recall_score_weighted': 0.8562192393736018, 'f1_score_micro': 0.8562192393736018, 'precision_score_weighted': 0.8578941203348339, 'precision_score_micro': 0.8562192393736018, 'norm_macro_recall': 0.6251815451270039, 'average_precision_score_micro': 0.9079934592326853, 'AUC_weighted': 0.8972312329186993, 'weighted_accuracy': 0.8901464643136598, 'recall_score_macro': 0.812590772563502, 'average_precision_score_weighted': 0.9048137088143138, 'log_loss': 0.39232972823397916, 'balanced_accuracy': 0.812590772563502, 'recall_score_micro': 0.8562192393736018, 'average_precision_score_macro': 0.8831261522703814, 'precision_score_macro': 0.853197651739741, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_6a6a1e12-6da0-4973-93fc-74bf858a4609_48

In [14]:
#TODO: Save the best model
best_run.register_model(model_name = "best_run_automl.pkl", model_path = './')
print(best_run)

Run(Experiment: heart-automl,
Id: AutoML_6a6a1e12-6da0-4973-93fc-74bf858a4609_48,
Type: azureml.scriptrun,
Status: Completed)


In [27]:
best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

model = remote_run.register_model(model_name = 'best_run_automl.pkl')
print(remote_run.model_id)

# https://knowledge.udacity.com/questions/463620

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

# Deploying the model via ACI WebService
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-deploy-azure-container-instance.md

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "heartfailure", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

best_run_automl.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-12 12:50:28+00:00 Creating Container Registry if not exists.
2022-03-12 12:50:28+00:00 Registering the environment.
2022-03-12 12:50:29+00:00 Use the existing image.
2022-03-12 12:50:29+00:00 Generating deployment configuration.
2022-03-12 12:50:30+00:00 Submitting deployment to compute.
2022-03-12 12:50:33+00:00 Checking the status of deployment heartfailure..
2022-03-12 12:53:22+00:00 Checking the status of inference endpoint heartfailure.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)

Service state: Healthy
Service scoring URI: http://003d3859-a205-4e86-b82b-3ecf7e405f21.southcentralus.azurecontainer.io/score
Service Swagger URI: http://003d3859-a205-4e86-b82b-3ecf7e405f21.southcentralus.azurecontainer.io/swagger.json
Service primary authentication key: FC31sRBImFyamhrh3VcbUU1FvBcKVMwa


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
%run endpoint.py

{"result": [1, 1]}
++++++++++++++++++++++++++++++
Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


In [25]:
print(service.get_logs())

2022-03-12T12:52:59,899765000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-03-12T12:52:59,910019000+00:00 - iot-server/run 
2022-03-12T12:52:59,909176900+00:00 - rsyslog/run 
2022-03-12T12:52:59,954964000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6797cf9b513e59b405ce80f3e9222a7d/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-03-12T12:53:00,401603000+00:00 - iot-server/finish 1 0
2022-03-12T12:53:00,408962300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (73)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-03-12 12:53:03,935 | root | INFO | Starting up app insights client
logging socket was

## Deleting the service
Putting the deletion of the service in a separate cell to avoid accidentally running the cell before finishing the tasks

In [28]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
